# Setup

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
import torch

`AutoTokenizer` służy do przetwarzania tekstu na format zrozumiały dla modelu. Zamienia on słowa i zdania na ciągi liczb (tokeny), które model może przetworzyć.

`AutoModelForCausalLM` to klasa odpowiedzialna za ładowanie modeli językownych typu przyczynowego (causal). Modele te przewidują kolejne tokeny na podstawie wcześniejszego kontekstu, co pozwala im generować spójny tekst.

`set_seed` to funkcja ustalająca ziarno losowości. Dzięki niej można uzyskać powtarzalne wyniki przy każdym uruchomieniu kodu.

In [22]:
class CFG:
    model = "Qwen/Qwen2-0.5B"

`"Qwen"` to rodzina modeli językowych stworzonych przez Alibaba Cloud. Liczba "2" wskazuje na drugą generację tego modelu. "0.5B" oznacza, że model ma około 500 milionów parametrów - są to wartości liczbowe, które model dostosowuje w trakcie uczenia się, aby lepiej rozumieć i generować tekst.

Rozmiar 500 milionów parametrów plasuje ten model w kategorii mniejszych modeli językowych. Dla porównania, modele takie jak GPT-3 mają setki miliardów parametrów. Mniejszy rozmiar ma jednak swoje zalety - model wymaga mniej zasobów obliczeniowych, może działać szybciej i być używany na słabszym sprzęcie.

# Tokenizacja

In [23]:
prompt = "It was a dark and stormy"
tokenizer = AutoTokenizer.from_pretrained(CFG.model)

`tokenizer = AutoTokenizer.from_pretrained(CFG.model)` Tokenizer działa jak tłumacz między ludzkim językiem a językiem modelu. Gdy wywołujemy metodę `from_pretrained`, następuje automatyczne pobranie odpowiedniego tokenizera z serwisu Hugging Face, gdzie przechowywane są modele i ich komponenty.

Tokenizer zamienia tekst na sekwencję tokenów - podstawowych jednostek, które model potrafi przetwarzać. Tokeny mogą być pojedynczymi słowami, częściami słów, lub nawet pojedynczymi znakami. Na przykład, zdanie "It was a dark and stormy" zostanie podzielone na tokeny w sposób specyficzny dla modelu Qwen. Tokenizer pamięta też, jak później połączyć tokeny z powrotem w tekst.

In [24]:
input_ids = tokenizer(prompt).input_ids

input_ids

[2132, 572, 264, 6319, 323, 13458, 88]

Najpierw tokenizer dzieli tekst na mniejsze części, czyli tokeny. Następnie każdemu tokenowi przypisuje unikalny numer identyfikacyjny (ID). Te identyfikatory są używane przez model, ponieważ pracuje on na liczbach, nie na tekście.

Gdy wywołujemy `tokenizer(prompt)`, otrzymujemy obiekt zawierający różne informacje o tokenizacji. Atrybut `.input_ids` wydobywa z tego obiektu konkretnie listę identyfikatorów numerycznych.

In [25]:
for t in input_ids:
    print(t, "\t: ", tokenizer.decode(t))

2132 	:  It
572 	:   was
264 	:   a
6319 	:   dark
323 	:   and
13458 	:   storm
88 	:  y


Następnie używamy metody `tokenizer.decode(t)`, która zamienia pojedynczy identyfikator z powrotem na tekst. To bardzo pouczające, ponieważ pokazuje nam, jak model "widzi" nasz tekst - jakie konkretnie fragmenty tekstu odpowiadają poszczególnym numerom.

# Prawdopodobienstwo

In [26]:
model = AutoModelForCausalLM.from_pretrained(CFG.model)

`AutoModelForCausalLM` to specjalna klasa z biblioteki transformers, która jest przeznaczona do modeli przyczynowych (causal). Określenie "przyczynowy" oznacza, że model generuje tekst sekwencyjnie, token po tokenie, gdzie każdy nowy token zależy tylko od poprzednich tokenów - podobnie jak człowiek piszący tekst, który widzi tylko to, co już napisał, nie to, co dopiero zamierza napisać.

Metoda `from_pretrained()` robi kilka ważnych rzeczy za kulisami. Po pierwsze, pobiera pliki modelu z repozytorium Hugging Face - to jak ściąganie specjalistycznego programu, który nauczył się rozumieć i generować tekst. W przypadku Qwen2-0.5B pobierane są pliki zawierające wszystkie parametry modelu - te 500 milionów liczb, które model wykorzystuje do przetwarzania języka.

In [27]:
input_ids = tokenizer(prompt, return_tensors = "pt").input_ids
outputs = model(input_ids)
outputs.logits.shape

torch.Size([1, 7, 151936])

`input_ids = tokenizer(prompt, return_tensors = "pt").input_ids` robi nieco więcej niż poprzednia tokenizacja, którą widzieliśmy.
* Parametr `return_tensors = "pt"` mówi tokenizerowi, żeby przekształcił wynik w format tensora PyTorch. To jak przekładanie tekstu nie tylko na liczby, ale na specjalny format matematyczny, który jest zoptymalizowany do obliczeń na GPU lub CPU. "pt" oznacza właśnie PyTorch - framework, którego używamy do obliczeń.

`outputs = model(input_ids)` to moment, gdy nasz tekst przechodzi przez model. To trochę jak przepuszczenie sygnału przez skomplikowany układ elektroniczny. Model przetwarza nasze tokeny, używając swoich 500 milionów parametrów, aby przewidzieć, jakie słowa mogłyby naturalnie wystąpić po naszym promptcie.

`outputs.logits.shape` pokazuje kształt (wymiary) wyjścia modelu. Wynik jest w postaci tak zwanych logitów - są to surowe wartości liczbowe przed przekształceniem ich w prawdopodobieństwa. Kształt tego tensora ma trzy wymiary:

1. Liczba sekwencji (batch size) - zazwyczaj 1, jeśli przetwarzamy pojedynczy prompt
2. Długość sekwencji - liczba tokenów w naszym inpucie
3. Rozmiar słownika - liczba wszystkich możliwych tokenów, które model zna

Dla każdej pozycji w naszej sekwencji model generuje wartość logitu dla każdego możliwego tokenu w swoim słowniku. Te wartości reprezentują "przekonanie" modelu o tym, jak dobrze każdy token pasowałby jako następny w sekwencji. Im wyższa wartość logitu, tym bardziej model "uważa", że dany token powinien wystąpić w tym miejscu.

In [28]:
final_logits = model(input_ids).logits[0,-1]
final_logits.argmax()

tensor(3729)

Te dwie linie kodu są kluczowym momentem w procesie generowania tekstu. Skupiają się na wyborze następnego tokenu w naszej sekwencji.

W linii `final_logits = model(input_ids).logits[0,-1]` wyodrębniamy konkretny fragment z wyjścia modelu. Rozbijmy indeksowanie `[0,-1]`:
- `0` wybiera pierwszą (i w tym przypadku jedyną) sekwencję z naszej paczki danych
- `-1` wybiera ostatnią pozycję w sekwencji - tam, gdzie model będzie przewidywał następny token

Te logity reprezentują "pewność" modelu co do każdego możliwego następnego tokenu. Można to porównać do muzyka, który zna wiele możliwych akordów i musi wybrać ten, który najlepiej pasuje do aktualnej melodii.

Druga linia `final_logits.argmax()` znajduje pozycję tokenu o najwyższej wartości logitu. Metoda `argmax()` to jak wskazanie palcem na najwyższy słupek w wykresie - zwraca indeks największej wartości. W kontekście naszego modelu językowego, ten indeks odpowiada tokenowi, który model uważa za najbardziej prawdopodobny jako następny w sekwencji.

To trochę jak wybieranie następnego słowa w grze w skojarzenia - model przeanalizował nasz prompt "It was a dark and stormy" i wskazuje token, który jego zdaniem najlepiej pasuje jako kontynuacja, bazując na wszystkim, czego nauczył się podczas treningu. Może to być na przykład słowo "night", które często występuje po frazie "dark and stormy", ale model mógł też wybrać coś bardziej zaskakującego, bazując na innych wzorcach, które rozpoznał w danych treningowych.

In [29]:
tokenizer.decode(final_logits.argmax())

' night'

In [30]:
top10_logits = torch.topk(final_logits, 10)

for index in top10_logits.indices:
    print(tokenizer.decode(index))

 night
 evening
 day
 morning
 winter
 afternoon
 Saturday
 Sunday
 Friday
 October


Pierwsza linia `top10_logits = torch.topk(final_logits,10)` używa funkcji `topk` z biblioteki PyTorch, która znajduje dziesięć najwyższych wartości w tensorze logitów. To jak patrzenie na dziesięć najsilniejszych intuicji modelu, zamiast tylko na tę najsilniejszą. Parametr 10 określa, ile najlepszych wyborów chcemy zobaczyć.

In [31]:
top10 = torch.topk(final_logits.softmax(dim=0), 10)

for value, index in zip(top10.values, top10.indices):
    print(f"{tokenizer.decode(index):<10}{value.item():.2%}")

 night    88.71%
 evening  4.30%
 day      2.19%
 morning  0.49%
 winter   0.45%
 afternoon0.27%
 Saturday 0.25%
 Sunday   0.19%
 Friday   0.17%
 October  0.16%


Pierwsza linia `top10 = torch.topk(final_logits.softmax(dim=0),10)` zawiera kluczową transformację -
* funkcję `softmax`. Jest to matematyczna operacja, która przekształca surowe logity (które mogą być dowolnymi liczbami) na prawdopodobieństwa, które zawsze sumują się do 100%.
  * Parametr `dim=0` określa, wzdłuż którego wymiaru ma być wykonana operacja softmax.

# Generowanie tekstu

In [32]:
output_ids = model.generate(input_ids, max_new_tokens = 20)
decoded_text = tokenizer.decode(output_ids[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [33]:
print("Input IDs", input_ids[0])
print("Output IDs", output_ids)
print(f"Generated text: {decoded_text}")

Input IDs tensor([ 2132,   572,   264,  6319,   323, 13458,    88])
Output IDs tensor([[ 2132,   572,   264,  6319,   323, 13458,    88,  3729,    13,   576,
         12884,   572,  6319,   323,   279,  9956,   572,  1246,  2718,    13,
           576, 11174,   572, 50413,  1495,   323,   279]])
Generated text: It was a dark and stormy night. The sky was dark and the wind was howling. The rain was pouring down and the


Te trzy linijki kodu pokazują nam pełny obraz procesu generowania tekstu - od początku do końca. Przeanalizujmy szczegółowo, co widzimy na każdym etapie.

`print("Input IDs", input_ids[0])` pokazuje numeryczne reprezentacje tokenów naszego początkowego tekstu "It was a dark and stormy". Jest to jak zerknięcie na "surowe dane", które model otrzymuje na wejściu. Każda liczba w tej sekwencji reprezentuje konkretny token, który model rozpoznaje jako część swojego słownika.

`print("Output IDs", output_ids)` przedstawia pełną sekwencję tokenów po generacji. To, co zobaczymy, to połączenie oryginalnych tokenów wejściowych oraz nowych tokenów, które model wygenerował. Jest to szczególnie interesujące, ponieważ możemy zobaczyć, jak model rozbudował oryginalną sekwencję. Pierwsze liczby będą identyczne z input_ids, a następnie pojawią się nowe liczby reprezentujące wygenerowaną kontynuację.

`print(f"Generated text: {decoded_text}")` pokazuje nam końcowy rezultat w formie czytelnej dla człowieka. Jest to moment transformacji - wszystkie numeryczne tokeny zostają przekształcone z powrotem w tekst. Zobaczymy nasz oryginalny prompt "It was a dark and stormy" wraz z wygenerowaną kontynuacją.

Porównanie tych trzech wyjść jest niezwykle pouczające. Możemy zaobserwować, jak ten sam tekst jest reprezentowany na różnych poziomach abstrakcji:
- jako sekwencja liczb, którymi operuje model
- jako rozszerzona sekwencja zawierająca nowe tokeny
- jako zrozumiały tekst w języku naturalnym

Jest to jak obserwowanie procesu tłumaczenia między trzema różnymi językami: językiem komputera (ID tokenów), językiem modelu (rozszerzona sekwencja) i językiem ludzkim (tekst). Każda reprezentacja niesie te same informacje, ale w formie odpowiedniej dla różnych etapów przetwarzania.

In [34]:
attention_mask = input_ids.ne(tokenizer.pad_token_id).long()

beam_output = model.generate(
input_ids,
num_beams=5,
max_new_tokens=30,
attention_mask=attention_mask,
pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(beam_output[0]))

It was a dark and stormy night. The wind was howling, and the rain was pouring down. The sky was dark and gloomy, and the air was filled with the


### Tworzenie attention_mask
* `input_ids` → Jest to tensor z tokenami wejściowymi dla modelu (zakodowany tekst).
* `tokenizer.pad_token_id` → To ID tokena paddingu (<pad>), który jest używany do wyrównywania sekwencji w batchu.
* `input_ids.ne(tokenizer.pad_token_id)`
  * `.ne()` oznacza "not equal" (czyli !=), co tworzy maskę True dla wszystkich tokenów różnych od <pad>.
  * Tokeny `<pad>` będą miały False, a reszta True.
* .long() → Konwertuje wartości True/False na 1/0 (long() = int64)

co daje tensor:
```txt
[1, 1, 1, 0, 0]
```
jeśli tokeny input_ids wyglądają tak:
```txt
[101, 1023, 2045, 0, 0]
```
### Generowanie tekstu:
✅ **Co tu się dzieje?**  

- **input_ids** → Model dostaje tokeny wejściowe do przetworzenia.  

- **num_beams=5**  
  - Włącza beam search z 5 ścieżkami (model generuje 5 możliwych kontynuacji i wybiera najlepszą).  
  - To poprawia jakość tekstu (lepsze niż standardowy greedy decoding).  

- **max_new_tokens=30**  
  - Ogranicza generowanie do 30 nowych tokenów (nie licząc wejścia).  

- **attention_mask=attention_mask**  
  - Informuje model, na które tokeny zwracać uwagę, a które ignorować (pomaga unikać błędów związanych z `<pad>`).  

- **pad_token_id=tokenizer.eos_token_id**  
  - Określa token końcowy (`<eos>` jako `pad_token_id`), co pomaga uniknąć błędów w dekodowaniu.

In [38]:
beam_output = model.generate(
input_ids,
num_beams=5,
repetition_penalty=2.0,
max_new_tokens=38,
attention_mask=attention_mask,
pad_token_id=tokenizer.eos_token_id
)

markdown_output = "\n".join(f"- {tokenizer.decode(output)}" for output in beam_output)
print(markdown_output)

- It was a dark and stormy night. The sky was filled with thunder and lightning, and the wind howled in the distance. It was raining cats and dogs, and the streets were covered in puddles of water.
